In [ ]:
using LinearAlgebra, SparseArrays
using Arpack
import gmsh
gmsh.initialize()

include("FEM.jl")
using .FEM

In [ ]:
# Adatok
E = 2e5
ν = 0.3
ρ = 7.85e-9
thickness = 1
height = 10
base = 100
elemSize = 2 #22

approxOrder = 5
internalNodes = true
quadElements = true

problem = FEM.Problem("rectangle", type="PlaneStress", E=E, ν=ν, ρ=ρ, b=thickness)

In [ ]:
# Geometria megrajzolása
gmsh.model.add(problem.name)

p1 = gmsh.model.occ.addPoint(0, 0, 0)
p2 = gmsh.model.occ.addPoint(base, 0, 0)
#p23 = gmsh.model.occ.addPoint(base, height / 2, 0)
p3 = gmsh.model.occ.addPoint(base, height, 0)
p4 = gmsh.model.occ.addPoint(0, height, 0)

l1 = gmsh.model.occ.addLine(p1, p2)
l2 = gmsh.model.occ.addLine(p2, p3)
#l23 = gmsh.model.occ.addLine(p23, p3)
l3 = gmsh.model.occ.addLine(p3, p4)
l4 = gmsh.model.occ.addLine(p4, p1)

cl1 = gmsh.model.occ.addCurveLoop([l1, l2, l3, l4])

l5 = gmsh.model.occ.addCircle(base / 2, height / 2, 0, min(base, height) / 4)
cl2 = gmsh.model.occ.addCurveLoop([l5])

sf1 = gmsh.model.occ.addPlaneSurface([cl1, cl2])
#display(sf1)

#cp1 = gmsh.model.occ.addPoint(base / 2, height / 2, 0)
#cl1 = gmsh.model.occ.addLine(p1, p3)
#cl2 = gmsh.model.occ.addLine(p2, p4)

#sf2 = gmsh.model.occ.fragment([(2, sf1)], [(1, cl1)])
#gmsh.model.occ.removeAllDuplicates()

#p5 = gmsh.model.occ.addPoint(0, height / 2, 0)
#p6 = gmsh.model.occ.addPoint(base, height / 2, 0)
#l5 = gmsh.model.occ.addLine(p5, p6)

gmsh.model.occ.synchronize()
#sf2

In [ ]:
phg = gmsh.model.addPhysicalGroup(1, [l2])
gmsh.model.setPhysicalName(1, phg, "supp")
phg = gmsh.model.addPhysicalGroup(1, [l4])
gmsh.model.setPhysicalName(1, phg, "load")

In [ ]:
supp = FEM.displacementConstraint("supp", ux=0, uy=0)
load = FEM.traction("load", fx=1, fy=0, thickness=thickness)

In [ ]:
FEM.generateMesh(problem, sf1, elemSize, approxOrder=approxOrder, algorithm=6, quadrangle=quadElements, internalNodes=internalNodes)

In [ ]:
ENV["LIBGL_ALWAYS_SOFTWARE"] = "true"
gmsh.fltk.run()
#gmsh.fltk.wait(3)

In [ ]:
# Néhány láthatósági beállítás 0:kikapcs. 1:bekapcs.
gmsh.option.setNumber("Geometry.CurveLabels", 0)
gmsh.option.setNumber("Geometry.Points", 0)
gmsh.option.setNumber("Geometry.Curves", 0)
gmsh.option.setNumber("Mesh.Points", 0)
gmsh.option.setNumber("Mesh.Lines", 0)
gmsh.option.setNumber("Mesh.SurfaceEdges", 0)
gmsh.option.setNumber("Mesh.NodeLabels", 0)
gmsh.option.setNumber("Mesh.LineLabels", 0)
gmsh.option.setNumber("Mesh.SurfaceLabels", 0)

In [ ]:
K = FEM.stiffnessMatrixPlaneStress(problem)

In [ ]:
M = FEM.massMatrixPlaneStress(problem)

In [ ]:
C = 1e-8 * K

In [ ]:
K, M, C, f = FEM.applyBoundaryConditions!(problem, K, M, C, [supp], [load]);

In [ ]:
Tₘᵢₙ = FEM.smallestPeriodTime(K, M)

In [ ]:
q = FEM.solveDisplacement(K, f)

$$\mathbf{M}\ddot{\mathbf{q}}+\mathbf{C}\dot{\mathbf{q}}+\mathbf{K}\mathbf{q}=\mathbf{f}$$
$$\mathbf{M}\ddot{\mathbf{q}}_n+\mathbf{C}\dot{\mathbf{q}}_n+\mathbf{K}\mathbf{q}_n=\mathbf{f}_n$$
$$\ddot{\mathbf{q}}_n = \frac{\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1}}{\Delta t^2}$$
$$\dot{\mathbf{q}}_n = \frac{\mathbf{q}_{n}-\mathbf{q}_{n-1}}{\Delta t}$$
$$\mathbf{M}\frac{\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1}}{\Delta t^2}+\mathbf{C}\frac{\mathbf{q}_{n}-\mathbf{q}_{n-1}}{\Delta t}+\mathbf{K}\mathbf{q}_n=\mathbf{f}_n$$
$$\mathbf{M}(\mathbf{q}_{n+1}-2\mathbf{q}_{n}+\mathbf{q}_{n-1})+\Delta t\mathbf{C}(\mathbf{q}_{n}-\mathbf{q}_{n-1})+\Delta t^2\mathbf{K}\mathbf{q}_n=\Delta t^2\mathbf{f}_n$$
$$\mathbf{M}\mathbf{q}_{n+1}=\left(2\mathbf{M}-\Delta t\mathbf{C}-\Delta t^{2}\mathbf{K}\right)\mathbf{q}_{n}+(\Delta t\mathbf{C}-\mathbf{M})\mathbf{q}_{n-1}+\Delta t^{2}\mathbf{f}_{n}$$

In [ ]:
c = √(problem.E / problem.ρ)
ξₘₐₓ = 1e-1
β = ξₘₐₓ * Tₘᵢₙ / π
C = β * K
u, v, t = FEM.CDM(K, M, C, base / c * 2, Tₘᵢₙ / π * (√(1 + ξₘₐₓ^2) - ξₘₐₓ) * 0.9)
size(u)

In [ ]:
S = FEM.solveStressPlaneStress(problem, q)

In [ ]:
gmsh.option.setNumber("View.IntervalsType", 3)
gmsh.option.setNumber("View.VectorType", 5)
gmsh.option.setString("View.Format", "%.6g")

In [ ]:
uvec = FEM.showResultUvec(problem, q, name="uvec", visible=false)
display("uvec done")
ux = FEM.showResultUX(problem, q, name="ux", visible=false)
display("ux done")
uy = FEM.showResultUY(problem, q, name="uy", visible=false)
display("uy done")
s = FEM.showResultS(problem, S, name="σ", visible=false, smooth=false)
display("s done")
sx = FEM.showResultSX(problem, S, name="σx", visible=false, smooth=false)
display("sx done")
sy = FEM.showResultSY(problem, S, name="σy", visible=false, smooth=false)
display("sy done")
sxy = FEM.showResultSXY(problem, S, name="τxy", visible=false, smooth=false)
display("sxy done")
#vvec = FEM.showResultsVTvec(problem, v, t, name="v(t)", visible=true)
display("vvec done")
#vvec = FEM.showResultUvec(problem, v[:, 1000], name="v(500)", visible=true)

In [ ]:
pp1 = gmsh.model.occ.addPoint(0, height / 2, 0)
pp2 = gmsh.model.occ.addPoint(base, height / 2, 0)
pp3 = gmsh.model.occ.addPoint(base / 8, height / 4, 0)

pl1 = gmsh.model.occ.addLine(pp1, pp2)
pl2 = gmsh.model.occ.addSpline([pp1, pp3, pp2])


gmsh.model.occ.synchronize()

In [ ]:
sts = ceil(Int64, (base / c * 2) / 6 / (Tₘᵢₙ / π * (√(1 + ξₘₐₓ^2) - ξₘₐₓ)))
display(sts)
Sp = FEM.solveStressPlaneStress(problem, u[:, sts])
sp = FEM.showResultS(problem, Sp, name="σ at t", visible=false, smooth=false)
FEM.plotOnPath(problem, pl1, sp, 500, name="σred on path", visible=false);

In [ ]:
#gmsh.option.setNumber("PostProcessing.Binary", 1)
#gmsh.option.setNumber("PostProcessing.SaveMesh", 1)

#gmsh.write("rect.msh")
#gmsh.write("rect.opt")
#gmsh.view.write(4, "rect.pos")

In [ ]:
ENV["LIBGL_ALWAYS_SOFTWARE"] = "false"
gmsh.fltk.run()

In [ ]:
gmsh.finalize()